In [1]:
#!/usr/bin/env python3
"""
PVDF-基板 界面構築・最適化プログラム (V4.3 - H2O密度指定挿入機能)

指定された基板ファイル (Al2O3, AlF3 など) を読み込み、
ジョブ設定に応じて「スーパーセル化」「サイズ指定クロップ」「絶対座標クロップ」の
いずれかを行い、
オプションで基板単体の最適化を実行した後、PVDFとの界面を構築。
★ 隙間にH2O分子を指定密度(例: 1 g/cm³)で挿入し、最後に全体を最適化する。
"""

import numpy as np
import os
from pathlib import Path
import time

# --- ASE (Atomic Simulation Environment) ---
from ase import Atoms, units # ★ units をインポート
# from ase.units import Avogadro # ★ Avogadro定数をインポート <- 削除 (mol を使う)
from ase.visualize import view
from ase.io import read, Trajectory, write
from ase.constraints import FixAtoms
from ase.build import molecule # ★ H2O のために追加

# --- PFP (Matlantis) ---
try:
    from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
    from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
    from matlantis_features.atoms import MatlantisAtoms
    from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
    from matlantis_features.utils.calculators import pfp_estimator_fn
    
    # 計算器初期化
    estimator = Estimator(calc_mode='CRYSTAL_U0_PLUS_D3')
    calculator = ASECalculator(estimator)
    MATLANTIS_AVAILABLE = True
    print("✅ PFP/Matlantis計算器を正常に初期化しました")
    
except ImportError as e:
    print(f"❌ PFP/Matlantisが利用できません: {e}")
    print("    最適化はスキップされます")
    calculator = None
    MATLANTIS_AVAILABLE = False


class PVDFInterfaceBuilder:
    """
    PVDFと基板の界面を構築・最適化するクラス
    (基板のオンザフライ・クロッピング機能を含む)
    """
    
    def __init__(self, base_dir="/home/jovyan/Kaori/MD/LiB_2"):
        self.base_dir = Path(base_dir)
        self.structure_dir = self.base_dir / "structure"
        
        # 基板ファイル (Al2O3_cell.cif など) が置かれる場所
        self.substrate_input_dir = self.structure_dir
        
        # 共通のPVDFファイルが置かれる場所
        self.pvdf_input_dir = self.structure_dir
        
        # 最終的な界面ファイルの出力先
        self.output_dir = self.structure_dir / "output" / "PVDF_interfaces"
        
        # 出力ディレクトリ作成
        self.output_dir.mkdir(parents=True, exist_ok=True)
        print(f"✅ システム初期化完了")
        print(f"    基板入力ディレクトリ: {self.substrate_input_dir}")
        print(f"    PVDF入力ディレクトリ: {self.pvdf_input_dir}")
        print(f"    界面出力ディレクトリ: {self.output_dir}")

    def run_matlantis_optimization(self, atoms, trajectory_path, fmax=0.05, name="structure"):
        """
        Matlantis最適化（元のスクリプトから流用）
        
        ★ V4.1 変更点 ★
        - KeyError: '962' 対策として、CIF保存の直前に atoms.info をクリアする処理を追加。
        - 軌跡(.traj)は保存しない設定 (FireLBFGSASEOptFeature の trajectory 引数がコメントアウトされている)
        - 最終構造の .xyz と .cif をこの関数内で保存する。
        """
        print(f"    -> Matlantis最適化開始 ({name}, fmax = {fmax}) ...")
        
        if not MATLANTIS_AVAILABLE:
            print(f"    -> ⚠️ Matlantisが利用できないため、最適化をスキップします")
            return atoms
        
        try:
            matlantis_atoms = MatlantisAtoms(atoms)
            estimator_function = pfp_estimator_fn(
                model_version='v7.0.0', calc_mode=EstimatorCalcMode.CRYSTAL_PLUS_D3
            )
            position_optimizer = FireLBFGSASEOptFeature(
                estimator_fn=estimator_function, filter=False,  
                # trajectory=str(trajectory_path), # 軌跡ファイル (必要ならコメント解除)
                n_run=5000, fmax=fmax, show_progress_bar=True
            )
            
            result = position_optimizer(matlantis_atoms)
            optimized_atoms = result.atoms.ase_atoms
            final_energy = result.output.energy_log[-1]
            print(f"    -> ✔️ 最適化完了！ エネルギー: {final_energy:.3f} eV")
            
            # --- 最終構造の保存 (XYZ/CIF) ---
            # trajectory_path の拡張子を .traj から .xyz/.cif に変更して使用
            xyz_save_path = Path(trajectory_path).with_suffix('.xyz')
            cif_save_path = Path(trajectory_path).with_suffix('.cif')
            
            # 1. XYZ形式で保存 (info は参照しないので問題なし)
            print(f"    -> {name} のXYZを保存: {xyz_save_path.name}")
            write(str(xyz_save_path), optimized_atoms)
            
            # 2. CIF形式で保存 (Matlantisのinfoを削除)
            #    (V4.1変更) .xyzを一度読み直して、Matlantis由来の .info を消去する
            atoms_for_cif = read(str(xyz_save_path))
            print(f"    -> {name} のCIFを保存: {cif_save_path.name}")
            write(str(cif_save_path), atoms_for_cif)
            
            return optimized_atoms
            
        except Exception as e:
            print(f"    -> ❌ 最適化中にエラーが発生: {e}")
            return None

            
    def _load_and_crop_substrate(self,  
                                     substrate_file_path: Path,  
                                     job_settings: dict, # ★ job_settings を受け取る
                                     vacuum: float = 15.0) -> Atoms:
        """
        基板ファイルを読み込み、ジョブ設定に応じて処理を行う。
        (優先順位: repeat_cell > crop_box_abs > crop_size)
        
        ★ V4.1 変更点 ★
        - 各分岐の最後に job_settings["optimize_substrate"] をチェックし、
          True なら基板単体の最適化 (run_matlantis_optimization) を実行する。
        """
        
        # 1. 構造ファイルを読み込み
        atoms_loaded = read(str(substrate_file_path))
        print(f"        読み込み原子数: {len(atoms_loaded)}")
        
        # 2. ジョブ設定に応じて処理を分岐
        repeat_cell = job_settings.get("repeat_cell", None)
        crop_box_abs = job_settings.get("crop_box_abs", None)
        crop_size = job_settings.get("crop_size", None)
        
        # 処理後の Atoms オブジェクトを格納する変数
        processed_atoms = None

        # --- 分岐A: スーパーセル化 (repeat_cell) ---
        if repeat_cell:
            if not (isinstance(repeat_cell, (list, tuple)) and len(repeat_cell) == 3):
                raise ValueError(f"repeat_cell は (nx, ny, nz) のタプルまたはリストで指定してください: {repeat_cell}")
            
            print(f"        -> スーパーセル化実行: {repeat_cell}")
            processed_atoms = atoms_loaded.repeat(repeat_cell)
            print(f"        -> スーパーセル化後の原子数: {len(processed_atoms)}")

        # --- 分岐B: 原点からのサイズでクロップ (crop_size) ---
        elif crop_size:
            if not (isinstance(crop_size, (list, tuple)) and len(crop_size) == 3):
                raise ValueError(f"crop_size は (x_size, y_size, z_size) のタプルまたはリストで指定してください: {crop_size}")
            
            x_size, y_size, z_size = crop_size
            # crop_box_abs に変換
            crop_box_abs = (0.0, x_size, 0.0, y_size, 0.0, z_size)
            print(f"        -> crop_size を crop_box_abs に変換: {crop_box_abs}")
            # このまま分岐Cの処理に流す

        # --- 分岐C: 絶対座標でクロップ (crop_box_abs) ---
        # (分岐Bから来た場合もここが実行される)
        if crop_box_abs and not repeat_cell: # repeat_cell が優先
            print(f"        -> クロップ実行 (X): {crop_box_abs[0]} - {crop_box_abs[1]} Å")
            print(f"        -> クロップ実行 (Y): {crop_box_abs[2]} - {crop_box_abs[3]} Å")
            print(f"        -> クロップ実行 (Z): {crop_box_abs[4]} - {crop_box_abs[5]} Å")

            pos = atoms_loaded.get_positions()
            x_min, x_max, y_min, y_max, z_min, z_max = crop_box_abs
            
            mask_x = (pos[:, 0] >= x_min) & (pos[:, 0] < x_max)
            mask_y = (pos[:, 1] >= y_min) & (pos[:, 1] < y_max)
            mask_z = (pos[:, 2] >= z_min) & (pos[:, 2] < z_max)
            
            final_mask = mask_x & mask_y & mask_z
            indices_to_keep = np.where(final_mask)[0]
            
            if len(indices_to_keep) == 0:
                print("        -> ⚠️ 警告: 指定された範囲に原子が見つかりません。")
                cropped_atoms = Atoms()
            else:
                cropped_atoms = atoms_loaded[indices_to_keep]
            
            print(f"        -> 切り出し後原子数: {len(cropped_atoms)}")

            # 4. 原子を (0, 0, 0) 基点に移動
            if len(cropped_atoms) > 0:
                cropped_atoms.positions[:, 0] -= x_min
                cropped_atoms.positions[:, 1] -= y_min
                z_min_cropped = cropped_atoms.positions[:, 2].min()
                cropped_atoms.positions[:, 2] -= z_min_cropped
            
            # 5. 新しい直交セルを設定
            new_cell = np.zeros((3, 3))
            new_cell[0, 0] = x_max - x_min
            new_cell[1, 1] = y_max - y_min
            
            if len(cropped_atoms) > 0:
                z_max_cropped = cropped_atoms.positions[:, 2].max()
                new_cell[2, 2] = z_max_cropped + vacuum
            else:
                new_cell[2, 2] = vacuum
            
            cropped_atoms.set_cell(new_cell)
            cropped_atoms.set_pbc([True, True, True])
            
            cell_params = cropped_atoms.get_cell_lengths_and_angles()
            print(f"        -> 新セル (直交): a={cell_params[0]:.2f}, b={cell_params[1]:.2f}, c={cell_params[2]:.2f}")
            
            processed_atoms = cropped_atoms

        # --- 分岐D: 何も指定がない場合 ---
        if processed_atoms is None:
            print("        -> クロップ/リピート指定なし。元の構造を使用します。")
            processed_atoms = atoms_loaded
            # セルが直交でない場合があるため警告
            cell_params = processed_atoms.get_cell_lengths_and_angles()
            if not (np.isclose(cell_params[3], 90) and np.isclose(cell_params[4], 90) and np.isclose(cell_params[5], 90)):
                print(f"        -> 警告: 基板セルの角度が直交(90, 90, 90)ではありません (gamma={cell_params[5]:.1f})。")
                print("        -> PVDF(gamma=60)とのスタックが意図通りにならない可能性があります。")

        # --- ★★★ V4.1 基板の事前最適化 ★★★ ---
        # (分岐A, C, D のいずれかを通過した後、ここで最適化を実行)
        if job_settings.get("optimize_substrate", False):
            print(f"        -> 基板の事前最適化を実行します...")
            
            # 最適化用の名前とパスを生成 (例: PVDF_on_Al2O3_cell_repeat_2x2x1_pre_opt)
            substrate_opt_name = job_settings.get("output_name", "substrate") + "_pre_opt"
            # .traj は run_matlantis_optimization が .cif/.xyz に変換するためのベース名として使用
            traj_path = self.output_dir / f"{substrate_opt_name}.traj" 
            
            # 基板最適化時は原子を固定しない (既存の制約を一時的に解除)
            original_constraint = processed_atoms.constraints
            processed_atoms.set_constraint(None) 
            
            optimized_substrate = self.run_matlantis_optimization(
                processed_atoms, traj_path, fmax=0.05, name=substrate_opt_name
            )
            
            if optimized_substrate:
                print(f"        -> ✔️ 基板の事前最適化 完了。")
                # 元の制約を戻す (通常はNoneだが念のため)
                optimized_substrate.set_constraint(original_constraint)
                return optimized_substrate
            else:
                print(f"        -> ❌ 基板の事前最適化に失敗。最適化前の基板を使用します。")
                # 元の制約を戻す
                processed_atoms.set_constraint(original_constraint)
                return processed_atoms
        
        else:
            # 事前最適化が指定されていない場合は、処理後の基板をそのまま返す
            return processed_atoms


    def build_pvdf_interface(self,  
                             top_pvdf_file: Path,  
                             job_settings: dict, # ★ 引数はジョブ辞書のまま
                             repeat_top: tuple = (2, 2, 1),
                             separation: float = 2.5,
                             vacuum: float = 15.0,
                             search_step: float = 0.25,
                             # --- ★ V4.3 H2O設定 (密度指定) ★ ---
                             target_h2o_density_g_cm3: float = 1.0, # ★ nmol_h2o から変更
                             add_h2o: bool = False,
                             h2o_padding: float = 0.5, # 基板/PVDFからの最小Z距離
                             h2o_overlap_threshold: float = 0.5,
                             h2o_max_attempts_mol: int = 5000
                             ):
        """
        PVDF(上)と基板(下)をカット＆スタックして界面を構築する
        
        ★ V4.3 変更点 ★
        - H2Oの数を固定値(nmol_h2o)から密度(target_h2o_density_g_cm3)で
          自動計算するように変更。
        """
        
        # ジョブ設定を展開
        output_name = job_settings["output_name"] # main() で自動生成された名前
        substrate_file_name = job_settings["substrate_file_name"]
        
        print(f"\n--- 処理開始: {output_name} ---")
        
        try:
            # --- STEP 1: ファイル読み込み & 基板処理 (クロップ/リピート/事前最適化) ---
            print("    1. 構造ファイルを読み込み中...")
            slab_top_orig = read(str(top_pvdf_file))     # PVDF (gamma=60)
            
            substrate_file_path = self.substrate_input_dir / substrate_file_name
            if not substrate_file_path.exists():
                print(f"    ❌ 基板ファイルが見つかりません: {substrate_file_path}")
                raise FileNotFoundError(substrate_file_path)

            # ★ 基板の読み込みと処理(クロップ/リピート/事前最適化)を実行 ★
            slab_bottom = self._load_and_crop_substrate(
                substrate_file_path,
                job_settings=job_settings, # ★ job_settings を丸ごと渡す
                vacuum=vacuum # クロップ時の仮真空（セル設定用）
            )
            
            # --- STEP 2: 基板のXYサイズとZ方向の厚みを取得 ---
            cell_params_bottom = slab_bottom.get_cell_lengths_and_angles()
            L_x = cell_params_bottom[0]
            L_y = cell_params_bottom[1]
            
            # ★ 基板のZ方向の厚み（原子の最大座標）を取得 ★
            if len(slab_bottom) > 0:
                # 事前最適化により z_min が 0 でなくなっている可能性があるため、(max - min) に変更
                z_pos_substrate = slab_bottom.positions[:, 2]
                substrate_thickness = z_pos_substrate.max() - z_pos_substrate.min()
                
                # 厚みが0（単原子層など）の場合、最低限の厚みを保証する
                if np.isclose(substrate_thickness, 0.0):
                    substrate_thickness = 5.0 # 最低 5Å は切り出す
                    print(f"    ⚠️ 基板の厚みがほぼ0のため、PVDFのZカット厚みを {substrate_thickness:.1f} Å に設定します。")
            else:
                raise ValueError("Substrate atoms count is zero.")

            # クロップした場合、セルは必ず直交になっているはず
            if not np.isclose(cell_params_bottom[5], 90):
                 print(f"    ⚠️ 警告: 基板のgamma角度が90度ではありません (gamma={cell_params_bottom[5]:.1f})。")
                 print("        PVDFの切り出し(STEP 3)が意図通りにならない可能性があります。")

            print(f"    2. 基板のXYサイズ(カット領域)を決定: L_x={L_x:.2f} Å, L_y={L_y:.2f} Å")
            print(f"       基板のZ方向厚み(PVDFカットに使用): {substrate_thickness:.2f} Å")
            
            # --- STEP 3: PVDF(60°)をスーパーセル化 & 最高密度領域の検索・カット ---
            print(f"    3. PVDFを {repeat_top} でスーパーセル化し、最高密度の領域を検索・カットします")
            
            # スーパーセル化
            slab_top_supercell = slab_top_orig.repeat(repeat_top)
            print(f"        -> スーパーセル化後の原子数: {len(slab_top_supercell)}")
            
            pos_top_all = slab_top_supercell.get_positions()
            
            # PVDFのZ=minを基点とし、基板の厚み(substrate_thickness)までを切り出す
            z_min_pvdf = pos_top_all[:, 2].min()
            z_mask = (pos_top_all[:, 2] < (z_min_pvdf + substrate_thickness)) & (pos_top_all[:, 2] >= z_min_pvdf)
            
            valid_indices_z = np.where(z_mask)[0]  
            pos_top_filtered = pos_top_all[valid_indices_z] 
            
            if len(pos_top_filtered) == 0:
                print(f"        -> ⚠️ Zフィルタ({z_min_pvdf:.2f}-{z_min_pvdf + substrate_thickness:.2f}Å)で全原子が除外されました。")
                print(f"        -> (PVDFのZ範囲が {pos_top_all[:, 2].min():.2f} ～ {pos_top_all[:, 2].max():.2f} Å のためかもしれません)")
                raise Exception(f"PVDF Z-filter returned no atoms.")

            # --- 最高密度領域の検索 (スライディングウィンドウ) ---
            print(f"        -> (L_x={L_x:.2f}, L_y={L_y:.2f}) の窓で最高密度領域を検索中 (Step={search_step} Å)...")
            
            x_min_search = pos_top_filtered[:, 0].min()
            x_max_search = pos_top_filtered[:, 0].max()
            y_min_search = pos_top_filtered[:, 1].min()
            y_max_search = pos_top_filtered[:, 1].max()

            x_offsets = np.arange(x_min_search, x_max_search - L_x + search_step, search_step)
            y_offsets = np.arange(y_min_search, y_max_search - L_y + search_step, search_step)

            if len(x_offsets) == 0 or len(y_offsets) == 0:
                 print(f"        -> ⚠️ 検索範囲が基板サイズより小さいです。")
                 print(f"        -> PVDFスーパーセル(原子占有): X={x_max_search-x_min_search:.1f}, Y={y_max_search-y_min_search:.1f}")
                 print(f"        -> 基板: X={L_x:.1f}, Y={L_y:.1f}")
                 print(f"        -> ★★★ 'repeat_top' の値を大きくしてください。 ★★★")
                 raise Exception("PVDF supercell is smaller than substrate.")

            best_offset = (0, 0)
            max_atom_count = 0
            best_indices_in_filtered = []  

            total_searches = len(x_offsets) * len(y_offsets)
            print_interval = max(1, total_searches // 10) 
            count = 0
            start_time_search = time.time()

            for x_off in x_offsets:
                for y_off in y_offsets:
                    count += 1
                    if count % print_interval == 0 or count == 1:
                        print(f"                 ...検索中 ({count}/{total_searches})")

                    win_x_min, win_x_max = x_off, x_off + L_x
                    win_y_min, win_y_max = y_off, y_off + L_y
                    
                    mask_in_window = (
                        (pos_top_filtered[:, 0] >= win_x_min) & (pos_top_filtered[:, 0] < win_x_max) &
                        (pos_top_filtered[:, 1] >= win_y_min) & (pos_top_filtered[:, 1] < win_y_max)
                    )
                    
                    current_atom_count = np.count_nonzero(mask_in_window)
                    
                    if current_atom_count > max_atom_count:
                        max_atom_count = current_atom_count
                        best_offset = (x_off, y_off)
                        best_indices_in_filtered = np.where(mask_in_window)[0]

            end_time_search = time.time()
            print(f"        -> ✔️ 検索完了 ({end_time_search - start_time_search:.1f} 秒)")
            print(f"        -> 最高密度: {max_atom_count} 原子")
            print(f"        -> 最適オフセット: (x={best_offset[0]:.2f}, y={best_offset[1]:.2f})")

            # --- 切り抜きとオフセット調整 ---
            indices_to_keep = valid_indices_z[best_indices_in_filtered]
            slab_top_cut = slab_top_supercell[indices_to_keep]
            slab_top_cut.positions[:, 0] -= best_offset[0]
            slab_top_cut.positions[:, 1] -= best_offset[1]
            slab_top = slab_top_cut
            print(f"        -> カット後のPVDF原子数: {len(slab_top)}")

            # --- STEP 4: スタック (積み重ね) ---
            print(f"    4. 2つのスラブを {separation} Å 離して積み重ねます")
            
            # 基板のZ座標の最大値を取得 (事前最適化で 0 基点ではなくなっているため)
            z_bottom_max = slab_bottom.positions[:, 2].max()
            
            # PVDFのZ座標の最小値を取得
            z_top_min = slab_top.positions[:, 2].min()
            
            # PVDFを基板の上に、separation の距離だけ離して配置
            slab_top.positions[:, 2] += (z_bottom_max - z_top_min + separation)
            
            interface = slab_bottom + slab_top
            print(f"        -> 結合後の総原子数: {len(interface)}")


            # --- ★★★ STEP 4.5: H2O 分子の挿入 ★★★ ---
            if add_h2o and target_h2o_density_g_cm3 > 0:
                print(f"    4.5. H2O 分子を 密度 {target_h2o_density_g_cm3} g/cm³ で隙間に挿入します")
                
                # --- H2O分子数の計算 ---
                # 挿入空間の体積 (Å^3)
                z_height_h2o = separation - (2.0 * h2o_padding)
                if z_height_h2o <= 0:
                    print(f"        -> ⚠️ H2O配置領域の厚みがありません (separation={separation}, padding={h2o_padding})。")
                    volume_A3 = 0.0
                else:
                    volume_A3 = L_x * L_y * z_height_h2o # (Å^3)
                
                # 水の分子量 (g/mol)
                h2o_molar_mass_g_mol = 18.015
                
                # 水1分子あたりの体積 (Å^3) を計算
                # 密度 (g/cm^3) -> モル体積 (cm^3/mol)
                h2o_molar_volume_cm3_mol = h2o_molar_mass_g_mol / target_h2o_density_g_cm3
                # モル体積 (cm^3/mol) -> 1分子の体積 (cm^3/molecule)
                # ★ 'Avogadro' を 'units.mol' に修正
                h2o_vol_per_mol_cm3 = h2o_molar_volume_cm3_mol / units.mol 
                # 1分子の体積 (cm^3) -> (Å^3)
                h2o_vol_per_mol_A3 = h2o_vol_per_mol_cm3 * 1e24

                nmol_h2o_to_add = 0
                if h2o_vol_per_mol_A3 > 1e-6: # ゼロ除算防止
                    nmol_h2o_to_add = int(np.floor(volume_A3 / h2o_vol_per_mol_A3))
                
                print(f"        -> 挿入空間体積: {volume_A3:.2f} Å³ (L_x={L_x:.2f}, L_y={L_y:.2f}, H={z_height_h2o:.2f})")
                print(f"        -> 水1分子の体積: {h2o_vol_per_mol_A3:.2f} Å³ (at {target_h2o_density_g_cm3} g/cm³)")
                print(f"        -> ★ 自動計算されたH2O分子数: {nmol_h2o_to_add} 個 ★")
                
                if nmol_h2o_to_add == 0:
                    print(f"        -> 挿入するH2Oが0個のため、STEP 4.5 をスキップします。")

                else:
                    # H2O 分子を定義
                    mol_h2o = molecule('H2O')
                    mol_h2o.center()
                    
                    # 配置する Z 空間を定義 (スラブ上面 + padding から PVDF下面 - padding まで)
                    z_min_h2o = z_bottom_max + h2o_padding
                    z_max_h2o = z_bottom_max + separation - h2o_padding
                
                    print(f"        -> 配置 Z 範囲: {z_min_h2o:.2f} - {z_max_h2o:.2f} Å")
                    print(f"        -> 衝突閾値: {h2o_overlap_threshold} Å")
                    
                    # セル情報 (スタック前の基板セルを使用)
                    temp_cell = slab_bottom.get_cell()
                    z_max_expected = z_bottom_max + separation + substrate_thickness + 10.0 # 余裕を持たせる
                    temp_cell[2, 2] = z_max_expected
                    
                    interface.set_cell(temp_cell)
                    interface.set_pbc([True, True, True]) # (クロップ済みなら T,T,T のはず)

                    n_added_h2o = 0
                    n_total_attempts_h2o = 0
                    
                    # ★ nmol_h2o_to_add 回 試行
                    for _ in range(nmol_h2o_to_add): 
                        n_attempts_this_mol = 0
                        is_placed = False

                        while not is_placed and n_attempts_this_mol < h2o_max_attempts_mol:
                            
                            mol_copy = mol_h2o.copy()
                            
                            mol_copy.rotate(np.random.uniform(0, 360), 'z', center='COM')
                            mol_copy.rotate(np.random.uniform(0, 180), 'y', center='COM')
                            mol_copy.rotate(np.random.uniform(0, 360), 'z', center='COM')
                            
                            x_rand = np.random.uniform(0.0, L_x)
                            y_rand = np.random.uniform(0.0, L_y)
                            z_rand = np.random.uniform(z_min_h2o, z_max_h2o)
                            
                            com = mol_copy.get_center_of_mass()
                            offset = [x_rand - com[0], y_rand - com[1], z_rand - com[2]]
                            mol_copy.positions += offset
                            
                            mol_copy.set_cell(temp_cell)
                            mol_copy.wrap()
                            
                            # --- 衝突チェック ---
                            pos_new = mol_copy.get_positions()
                            pos_existing = interface.get_positions()
                            
                            min_dist = np.inf
                            for pos_new_atom in pos_new:
                                diff = pos_existing - pos_new_atom
                                
                                # ★ Z方向(i=2)はスラブなのでPBCを考慮しない
                                for i in range(2): # XY のみ
                                    if interface.pbc[i]:
                                        cell_length = temp_cell[i, i]
                                        if cell_length > 1e-6:
                                            diff[:, i] -= np.round(diff[:, i] / cell_length) * cell_length
                                
                                distances = np.linalg.norm(diff, axis=1)
                                min_dist_this_atom = distances.min()
                                
                                if min_dist_this_atom < min_dist:
                                    min_dist = min_dist_this_atom

                            # --- 判定 ---
                            if min_dist > h2o_overlap_threshold:
                                interface += mol_copy
                                n_added_h2o += 1
                                is_placed = True
                                if n_added_h2o % 10 == 0 or n_added_h2o == nmol_h2o_to_add:
                                    print(f"        -> H2O {n_added_h2o}/{nmol_h2o_to_add} 追加完了 (Min dist: {min_dist:.2f} Å)")
                            else:
                                n_attempts_this_mol += 1
                                n_total_attempts_h2o += 1
                        
                        if not is_placed:
                            print(f"        -> ⚠️ H2O 分子 {n_added_h2o + 1} が {h2o_max_attempts_mol} 回の試行で配置できませんでした。")
                            print(f"        -> 密度が高すぎる可能性があります。H2Oの配置を停止します。")
                            break # for ループを抜ける
                    
                    print(f"        -> ✔️ H2O 挿入完了。合計 {n_added_h2o} 個の H2O を追加。")
                    print(f"        -> 結合後の総原子数 (H2O含む): {len(interface)}")


            # --- STEP 5: 最終セルの設定 ---
            print(f"    5. 最終セルを設定します (基板のXY + 真空 {vacuum} Å)")
            
            # 基板のセル（クロップ後は直交のはず）をベースにする
            final_cell = slab_bottom.get_cell()
            
            # Z方向の真空層を設定 (全体のZ-minを0にシフト)
            z_min_total = interface.positions[:, 2].min()
            interface.positions[:, 2] -= z_min_total
            z_max_total = interface.positions[:, 2].max()
            final_cell[2, 2] = z_max_total + vacuum
            
            # 基板の最下層 (z < 3.0) の原子を固定
            # (z_min_total を引いた後の座標系で判断)
            indices = [i for i, z in enumerate(interface.positions[:, 2]) if z < 3.0]
            
            if len(indices) == 0:
                print("    警告: 固定原子がありません (z < 3.0 の条件)")
            else:
                print(f"        -> {len(indices)} 個の原子(z < 3.0)を固定します")
                
            constraint = FixAtoms(indices=indices)
            interface.set_constraint(constraint)
            interface.set_cell(final_cell)
            interface.pbc = [True, True, True]  
            
            # ★ H2O追加後に、ここで最終的なセルで wrap する
            interface.wrap() 
            
            write(str(self.output_dir / f"{output_name}_initial.xyz"), interface)

            # --- STEP 6: 界面の最適化 ---
            print("    6. 界面構造の最適化を実行します")
            # .traj は run_matlantis_optimization が .cif/.xyz に変換するためのベース名として使用
            traj_path = self.output_dir / f"{output_name}_final.traj" 
            
            optimized_interface = self.run_matlantis_optimization(
                interface, traj_path, fmax=0.05, name=output_name
            )

            # --- STEP 7: 最終構造の保存 ---
            # (run_matlantis_optimization 内で実行されるため、ここは不要)
            if optimized_interface:
                print(f"✅ 処理完了: {output_name}")
            else:
                print("❌ 最適化に失敗したため、最終ファイルを保存できませんでした")

        except Exception as e:
            print(f"❌ 界面 {output_name} の作成中に重大なエラーが発生: {e}")
            import traceback
            traceback.print_exc() # 詳細なエラーを表示
    

# ================== メイン実行 (連続処理モード) ==================
def main():
    print("=" * 70)
    print("🧪 PVDF-基板 界面構築・最適化システム (V4.3 - H2O密度指定挿入機能)")
    print("=" * 70)
    
    # --- 1. システム初期化 ---
    builder = PVDFInterfaceBuilder(base_dir="/home/jovyan/Kaori/MD/LiB_2")

    # --- 2. ★★★ 共通設定 ★★★ ---
    pvdf_file_name = "PVDF_only_shrunk.cif"
    repeat_top = (5, 5, 3) # PVDFスーパーセルサイズ (基板より十分大きく)
    separation_dist = 3.0  # 界面の距離 (Å) <- ★H2Oはこの隙間に入る
    vacuum_size = 0.1      # Z方向の真空層 (Å)
    search_step_size = 0.25 # 密度検索のステップ幅(Å)

    # --- ★★★ H2O 共通設定 (V4.3 密度指定) ★★★ ---
    ADD_H2O = True              # ★ 水を追加する場合は True にする
    TARGET_H2O_DENSITY_G_CM3 = 2.0 # ★ 目標密度 (g/cm^3)
    H2O_PADDING = 0.5           # ★ 基板表面/PVDF表面からの最小Z距離 (Å)
    # (例: separation=2.0, padding=0.5 の場合、1.0Å の厚みの空間に水が入る)

    # --- 3. ★★★ 処理対象リスト (クロップ/リピート/事前最適化 指定が可能) ★★★ ---
    #
    # "repeat_cell": (nx, ny, nz)
    # "crop_box_abs": (x_min, x_max, y_min, y_max, z_min, z_max)
    # "crop_size": (x_size, y_size, z_size)
    #
    # ★ V4.1 追加 ★
    # "optimize_substrate": True  (これを追加すると、基板単体を先に最適化します)
    #
    
    substrate_jobs = [
        # --- 例1: (2,2,1) スーパーセル化 + ★基板を事前最適化★ ---
        {
            "substrate_file_name": "Al2O3_cell.cif",
            # "repeat_cell": (1, 1, 1),
            "optimize_substrate":False # ★★★ 基板の事前最適化を実行 ★★★
        },

        # --- 例2: (2,2,1) スーパーセル化 (事前最適化なし) ---
        {
            "substrate_file_name": "AlF3_cell.cif",
            # "repeat_cell": (2, 2, 1)
            # "optimize_substrate": False (デフォルト)
        },

        # --- 例3: (2,2,1) スーパーセル化 (事前最適化なし) ---
        {
            "substrate_file_name": "Al_cell.cif",
            # "repeat_cell": (2, 2, 1)
        },
        
        # # --- 例4: (1,1,2) スーパーセル化 + ★基板を事前最適化★ ---
        # {
        #     "substrate_file_name": "Al2O3_cell.cif",
        #     "repeat_cell": (1, 1, 2),
        #     "optimize_substrate": True # ★★★ 基板の事前最適化を実行 ★★★
        # },

        # # --- 例5: (1,1,2) スーパーセル化 (事前最適化なし) ---
        # {
        #     "substrate_file_name": "AlF3_cell.cif",
        #     "repeat_cell": (1, 1, 2)
        # },

        # # --- 例6: (1,1,2) スーパーセル化 (事前最適化なし) ---
        # {
        #     "substrate_file_name": "Al_cell.cif",
        #     "repeat_cell": (1, 1, 2)
        # },
        
        # # --- 例7: (1,1,3) スーパーセル化 + ★基板を事前最適化★ ---
        # {
        #     "substrate_file_name": "Al2O3_cell.cif",
        #     "repeat_cell": (1, 1, 3),
        #     "optimize_substrate": True # ★★★ 基板の事前最適化を実行 ★★★
        # },

        # # --- 例8: (1,1,3) スーパーセル化 (事前最適化なし) ---
        # {
        #     "substrate_file_name": "AlF3_cell.cif",
        #     "repeat_cell": (1, 1, 3)
        # },

        # # --- 例9: (1,1,3) スーパーセル化 (事前最適化なし) ---
        # {
        #     "substrate_file_name": "Al_cell.cif",
        #     "repeat_cell": (1, 1, 3)
        # },
    ]

    # --- 4. 連続実行 ---
    print(f"▶️ {len(substrate_jobs)} 件の界面構築を連続で開始します")

    common_pvdf_path = builder.pvdf_input_dir / pvdf_file_name
    if not common_pvdf_path.exists():
        print(f"❌ 共通のPVDFファイルが見つかりません: {common_pvdf_path}")
        print("処理を中止します。")
        return

    # リストを順番に処理
    for i, job in enumerate(substrate_jobs, 1):
        
        # --- ★ output_name を自動生成 (ロジック拡張) ★ ---
        base_name = Path(job["substrate_file_name"]).stem # "Al2O3_cell.cif" -> "Al2O3_cell"
        
        repeat_cell = job.get("repeat_cell", None)
        crop_box_abs = job.get("crop_box_abs", None)
        crop_size = job.get("crop_size", None)
        
        suffix = ""

        if repeat_cell:
            suffix = f"_repeat_{repeat_cell[0]}x{repeat_cell[1]}x{repeat_cell[2]}"
        
        elif crop_box_abs:
            Lx = crop_box_abs[1] - crop_box_abs[0]
            Ly = crop_box_abs[3] - crop_box_abs[2]
            Lx_str = f"{Lx:.1f}".replace('.0', '').replace('.', '_')
            Ly_str = f"{Ly:.1f}".replace('.0', '').replace('.', '_')
            suffix = f"_cropped_abs_{Lx_str}x{Ly_str}"
        
        elif crop_size:
            Lx = crop_size[0]
            Ly = crop_size[1]
            Lx_str = f"{Lx:.1f}".replace('.0', '').replace('.', '_')
            Ly_str = f"{Ly:.1f}".replace('.0', '').replace('.', '_')
            suffix = f"_cropped_size_{Lx_str}x{Ly_str}"

        else:
            suffix = "_original_cell"
        
        # ★ H2O を追加する場合、ファイル名にサフィックスを追加 (V4.3 修正)
        h2o_suffix = ""
        if ADD_H2O and TARGET_H2O_DENSITY_G_CM3 > 0:
            # 密度が 1.0 以外の場合も考慮 (例: 0.997 -> 0_997)
            density_str = f"{TARGET_H2O_DENSITY_G_CM3}".replace('.', '_')
            h2o_suffix = f"_with_H2O_d{density_str}"

        output_name = f"PVDF_on_{base_name}{suffix}{h2o_suffix}"
        
        # 生成した output_name を job 辞書に追加
        job["output_name"] = output_name
        # --- ★ 自動生成ここまで ★ ---

        print("\n" + "=" * 50)
        print(f"       ({i}/{len(substrate_jobs)}) ジョブ開始: {job['output_name']}")
        print(f"       基板ファイル: {job['substrate_file_name']}")
        
        # V4.1: 事前最適化の有無を表示
        if job.get("optimize_substrate", False):
            print(f"       基板処理: {suffix.strip('_')} + ★事前最適化あり★")
        else:
            print(f"       基板処理: {suffix.strip('_')} (事前最適化なし)")
            
        # V4.3: H2Oの有無を表示
        if ADD_H2O and TARGET_H2O_DENSITY_G_CM3 > 0:
            print(f"       H2O挿入: ★あり★ (目標密度: {TARGET_H2O_DENSITY_G_CM3} g/cm³)")
        else:
            print(f"       H2O挿入: なし")


        print(f"=" * 50)
        
        # ★ 実行 ★
        builder.build_pvdf_interface(
            top_pvdf_file=common_pvdf_path,
            job_settings=job, # ★ ジョブ辞書を丸ごと渡す
            repeat_top=repeat_top,
            separation=separation_dist,
            vacuum=vacuum_size,
            search_step=search_step_size,
            # --- ★ H2O パラメータを渡す (V4.3 修正) ★ ---
            add_h2o=ADD_H2O,
            target_h2o_density_g_cm3=TARGET_H2O_DENSITY_G_CM3,
            h2o_padding=H2O_PADDING
        )

    print("\n" + "=" * 70)
    print("🎉 全てのジョブが完了しました！")
    print(f"📂 出力先: {builder.output_dir}")

if __name__ == "__main__":
    main()

✅ PFP/Matlantis計算器を正常に初期化しました
🧪 PVDF-基板 界面構築・最適化システム (V4.3 - H2O密度指定挿入機能)
✅ システム初期化完了
    基板入力ディレクトリ: /home/jovyan/Kaori/MD/LiB_2/structure
    PVDF入力ディレクトリ: /home/jovyan/Kaori/MD/LiB_2/structure
    界面出力ディレクトリ: /home/jovyan/Kaori/MD/LiB_2/structure/output/PVDF_interfaces
▶️ 3 件の界面構築を連続で開始します

       (1/3) ジョブ開始: PVDF_on_Al2O3_cell_repeat_2x2x1_with_H2O_d2_0
       基板ファイル: Al2O3_cell.cif
       基板処理: repeat_2x2x1 (事前最適化なし)
       H2O挿入: ★あり★ (目標密度: 2.0 g/cm³)

--- 処理開始: PVDF_on_Al2O3_cell_repeat_2x2x1_with_H2O_d2_0 ---
    1. 構造ファイルを読み込み中...
        読み込み原子数: 380
        -> スーパーセル化実行: (2, 2, 1)
        -> スーパーセル化後の原子数: 1520
    2. 基板のXYサイズ(カット領域)を決定: L_x=27.12 Å, L_y=23.26 Å
       基板のZ方向厚み(PVDFカットに使用): 18.89 Å
    3. PVDFを (5, 5, 3) でスーパーセル化し、最高密度の領域を検索・カットします
        -> スーパーセル化後の原子数: 111150
        -> (L_x=27.12, L_y=23.26) の窓で最高密度領域を検索中 (Step=0.25 Å)...
                 ...検索中 (1/168370)


/tmp/ipykernel_8729/1615971045.py:311: DeprecationWarning: Please use atoms.cell.cellpar() instead
  cell_params_bottom = slab_bottom.get_cell_lengths_and_angles()


                 ...検索中 (16837/168370)
                 ...検索中 (33674/168370)
                 ...検索中 (50511/168370)
                 ...検索中 (67348/168370)
                 ...検索中 (84185/168370)
                 ...検索中 (101022/168370)
                 ...検索中 (117859/168370)
                 ...検索中 (134696/168370)
                 ...検索中 (151533/168370)
                 ...検索中 (168370/168370)
        -> ✔️ 検索完了 (14.8 秒)
        -> 最高密度: 893 原子
        -> 最適オフセット: (x=40.48, y=13.32)
        -> カット後のPVDF原子数: 893
    4. 2つのスラブを 3.0 Å 離して積み重ねます
        -> 結合後の総原子数: 2413
    4.5. H2O 分子を 密度 2.0 g/cm³ で隙間に挿入します
        -> 挿入空間体積: 1261.39 Å³ (L_x=27.12, L_y=23.26, H=2.00)
        -> 水1分子の体積: 14.96 Å³ (at 2.0 g/cm³)
        -> ★ 自動計算されたH2O分子数: 84 個 ★
        -> 配置 Z 範囲: 19.39 - 21.39 Å
        -> 衝突閾値: 0.5 Å
        -> H2O 10/84 追加完了 (Min dist: 2.01 Å)
        -> H2O 20/84 追加完了 (Min dist: 1.82 Å)
        -> H2O 30/84 追加完了 (Min dist: 1.56 Å)
        -> H2O 40/84 追加完了 (Min dist: 0.99 Å)
        -

  0%|          | 0/5001 [00:00<?, ?it/s]

    -> ✔️ 最適化完了！ エネルギー: -13838.637 eV
    -> PVDF_on_Al2O3_cell_repeat_2x2x1_with_H2O_d2_0 のXYZを保存: PVDF_on_Al2O3_cell_repeat_2x2x1_with_H2O_d2_0_final.xyz
    -> PVDF_on_Al2O3_cell_repeat_2x2x1_with_H2O_d2_0 のCIFを保存: PVDF_on_Al2O3_cell_repeat_2x2x1_with_H2O_d2_0_final.cif
    -> ❌ 最適化中にエラーが発生: '962'
❌ 最適化に失敗したため、最終ファイルを保存できませんでした

       (2/3) ジョブ開始: PVDF_on_AlF3_cell_repeat_2x2x1_with_H2O_d2_0
       基板ファイル: AlF3_cell.cif
       基板処理: repeat_2x2x1 (事前最適化なし)
       H2O挿入: ★あり★ (目標密度: 2.0 g/cm³)

--- 処理開始: PVDF_on_AlF3_cell_repeat_2x2x1_with_H2O_d2_0 ---
    1. 構造ファイルを読み込み中...
        読み込み原子数: 270
        -> スーパーセル化実行: (2, 2, 1)
        -> スーパーセル化後の原子数: 1080
    2. 基板のXYサイズ(カット領域)を決定: L_x=22.48 Å, L_y=23.32 Å
       基板のZ方向厚み(PVDFカットに使用): 17.66 Å
    3. PVDFを (5, 5, 3) でスーパーセル化し、最高密度の領域を検索・カットします
        -> スーパーセル化後の原子数: 111150
        -> (L_x=22.48, L_y=23.32) の窓で最高密度領域を検索中 (Step=0.25 Å)...
                 ...検索中 (1/174032)
                 ...検索中 (17403/174032)
                 ...検索

  0%|          | 0/5001 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [2]:

# --- 1. インポート (重複をすべて削除し、クリーンアップ) ---
import numpy as np
import pandas as pd
import os
import time
import re
import logging
from time import perf_counter
from typing import Dict, Any
from pathlib import Path
from datetime import datetime

from ase.io import read, write, Trajectory
from ase.constraints import FixAtoms
from ase import units
from ase.build import add_vacuum

from sklearn.linear_model import LinearRegression

# --- Matlantis (PFP) のインポート ---
try:
    from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
    from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
    from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator as PFP # v7.0.0以降の推奨
    
    from matlantis_features.atoms import MatlantisAtoms
    from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
    from matlantis_features.utils.atoms_util import convert_atoms_to_upper
    
    from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
    
    # MDシミュレーションに必要なインポート
    from matlantis_features.features.md import (
        ASEMDSystem, 
        MDFeature, 
        MDExtensionBase,
        NVTBerendsenIntegrator, 
        NPTIntegrator, 
        NPTBerendsenIntegrator,
        LangevinIntegrator,
        VelocityVerletIntegrator # select_integratorで使う場合
    )
    from matlantis_features.features.md.md_extensions import DeformScheduler

except ImportError as e:
    print(f"Error: 必要なライブラリが見つかりません。- {e}")
    print("pfp-api-client (pfp-ase) と matlantis_features が正しくインストールされているか確認してください。")
    # エラーが発生した場合、ここで処理を停止させる
    raise e

# --- 2. MDヘルパー関数定義 ---

class PrintWriteLog(MDExtensionBase):
    """
    MDシミュレーションのログを記録・出力するクラス
    (定義は1回だけにします)
    """
    def __init__(self, fname: str, dirout: str = '.', stdout: bool = False):
        self.fname = fname
        self.dirout = dirout
        self.t_start = perf_counter()
        self.stdout = stdout

    def __call__(self, system, integrator):
        n_step = system.current_total_step
        sim_time = system.current_total_time / 1000    # ps
        E_tot = system.ase_atoms.get_total_energy()
        E_pot = system.ase_atoms.get_potential_energy()
        E_kin = system.ase_atoms.get_kinetic_energy()
        temp = system.ase_atoms.get_temperature()
        
        try:
            density = system.ase_atoms.get_masses().sum() / units.mol / (
                system.ase_atoms.cell.volume * (1e-8**3)
            )
        except Exception:
            density = 0.0 # 体積が0の場合などのエラー回避

        calc_time = (perf_counter() - self.t_start) / 60.  # min.

        # ヘッダー書き込み
        if n_step == 0:
            hdr = 'step,time[ps],E_tot[eV],E_pot[eV],E_kin[eV],'
            hdr += 'T[K],density[g/cm3],calc_time[min]'
            with open(f'{self.dirout}/{self.fname}.log', 'w') as f_log:
                f_log.write(f'{hdr}\n')

        # 結果の書き込みと出力
        line = f'{n_step:8d},{sim_time:7.2f},'
        line += f'{E_tot:11.4f},{E_pot:11.4f},{E_kin:9.4f},'
        line += f'{temp:8.2f},{density:7.3f},{calc_time:8.2f}'
        with open(f'{self.dirout}/{self.fname}.log', 'a') as f_log:
            f_log.write(f'{line}\n')
        if self.stdout:
            print(line)

# (注: select_integrator は元のコードでコメントアウトされていたため、ここでも除外します)

def run_md_simulation(
    atoms,
    integrator_type: str, # この引数は使われませんが、互換性のために残します
    temperature: float,
    n_steps: int,
    timestep: float = 1.0,
    pressure: float = 101325.0, # この引数も使われません
    traj_file: str = "md.traj",
    traj_freq: int = 50,
    logger_interval: int = 50,
    model_version: str = 'v7.0.0',
    calc_mode: str = 'CRYSTAL_U0',
    show_progress: bool = True,
    show_logger: bool = True,
):
    """
    MDシミュレーションを実行する汎用関数
    (定義は1回だけにします)
    (注: 元のコードに基づき、IntegratorはNVT_Berendsenに固定されています)
    """
    logger = logging.getLogger("matlantis_features")
    logger.setLevel(logging.INFO)
    estimator_fn = pfp_estimator_fn(model_version=model_version, calc_mode=calc_mode)
    
    # ASEMDSystemに渡すatomsがMatlantisAtomsであることを確認
    if not isinstance(atoms, MatlantisAtoms):
         atoms = MatlantisAtoms(atoms)
         
    system = ASEMDSystem(atoms)
    
    system.init_temperature(
        temperature=temperature,
        stationary=True,
        rng=np.random.RandomState(seed=12345)
    )
    print(f"システムを {temperature} K に初期化しました。")

    # 元のコードのロジックに基づき、NVT_Berendsenに固定
    integrator=NVTBerendsenIntegrator(
            timestep=timestep,
            temperature=temperature,
            taut=100.0 * units.fs
        )
    
    md = MDFeature(
        integrator=integrator,
        n_run=n_steps,
        show_progress_bar=show_progress,
        show_logger=show_logger,
        logger_interval=logger_interval,
        traj_file_name=traj_file,
        traj_freq=traj_freq,
        estimator_fn=estimator_fn,
    )
    
    print(f"\n--- MDシミュレーションを開始します ({n_steps} ステップ) ---")
    
    # ★★★ ここでシミュレーションが実行されます ★★★
    md_results = md(system)
    # ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
    
    print("\n--- MDシミュレーションが正常に完了しました ---")
    print(f"最終的なトラジェクトリは '{traj_file}' に保存されました。")
    print(f"最終ステップ: {system.current_total_step}, 最終時間: {system.current_time:.2f} fs")
    
    return md_results

# --- 3. メモリ解放 (計算実行前) ---
import gc
print("シミュレーション実行前にメモリを解放します...")
# 前のセルで定義された `optimized_atoms` などの巨大な変数を削除
# (もし `atoms` 変数にコピー済みで、`optimized_atoms` が不要な場合)
#
# try:
#     del optimized_atoms 
# except NameError:
#     pass # 変数が存在しない場合は何もしない

gc.collect() # ガベージコレクションを実行
print("メモリ解放完了。")


# --- 4. シミュレーションパラメータ設定 (重複を削除) ---

# (注: `INPUT_FILE` や `RELAX_TRAJ` など、このセルで使わないパラメータは削除しました)
# (注: `atoms` 変数は、前のセルで NPT緩和後の構造として定義されている前提です)

# OUTPUT_TRAJ = 'Al-23PVDF_md_H2O_1600K_fix_al.traj' # 出力トラジェクトリ

# --- 本番MD用のパラメータ ---
TARGET_TEMP_K = 500.0      # 論文の図21の条件 (2000 K)
TIME_STEP_FS = 0.1          # 論文の条件 (0.1 fs)
TOTAL_TIME_PS = 40.0       # 論文の条件 (100 ps)

# --- 5. シミュレーション実行 ---

total_steps = int((TOTAL_TIME_PS * 1000) / TIME_STEP_FS) # 1,000,000 ステップ

# ログとトラジェクトリへの書き込み頻度 (ステップ数)
# 1000ステップごと (0.1 fs * 1000 = 100 fs = 0.1 psごと)
log_interval = 1000

print(f"\nStarting NVT MD simulation at {TARGET_TEMP_K} K...")
print(f"Time step: {TIME_STEP_FS} fs")
print(f"Total steps: {total_steps} (for {TOTAL_TIME_PS} ps)")
print(f"Log/Traj interval: {log_interval} steps")
lists=[
"/home/jovyan/Kaori/MD/LiB_2/structure/output/PVDF_interfaces/PVDF_on_Al2O3_cell_repeat_2x2x1_with_H2O_d2_0_final.xyz",
"/home/jovyan/Kaori/MD/LiB_2/structure/output/PVDF_interfaces/PVDF_on_Al_cell_repeat_2x2x1__with_H2O_d2_0_final.xyz",
"/home/jovyan/Kaori/MD/LiB_2/structure/output/PVDF_interfaces/PVDF_on_AlF3_cell_repeat_2x2x1__with_H2O_d2_0_final.xyz"]
for list2 in lists:
    
    atoms=read(list2)
    OUTPUT_TRAJ = Path(list2).with_suffix('.traj') # 出力トラジェクトリ
    positions=atoms.get_positions()
    atoms.set_cell([17.18,17.18,positions[:,2].max()+1.0])
    atoms.set_pbc([True,True,True])
    fix_z_threshold = 6.0
    fix_indices = [atom.index for atom in atoms if atom.position[2] < fix_z_threshold]
    
    if fix_indices:
        print(f"z < {fix_z_threshold} Å にある {len(fix_indices)} 個の原子を固定します。")
        constraint = FixAtoms(indices=fix_indices)
        atoms.set_constraint(constraint)
    else:
        print(f"z < {fix_z_threshold} Å にある原子は見つかりませんでした。制約は設定されません。")
    # print(atoms.cell)
    # `atoms` 変数がこのセルの前に定義されていることを確認
    if 'atoms' not in locals():
        print("エラー: `atoms` 変数が定義されていません。")
        print("前のセルで NPT 緩和を実行し、その結果を `atoms` に代入してください。")
        # raise NameError("`atoms` is not defined.")
    else:
        # メインのMDシミュレーションを実行
        md_results = run_md_simulation(
            atoms=atoms, # NPT緩和後の構造を使用
            integrator_type="NVT_Berendsen", # (この引数は関数内で上書きされます)
            temperature=TARGET_TEMP_K,
            n_steps=total_steps,
            timestep=TIME_STEP_FS,
            traj_file=OUTPUT_TRAJ,
            traj_freq=log_interval,
            logger_interval=log_interval,
            model_version='v7.0.0',      # MDに使用するモデル
            calc_mode='CRYSTAL_U0',      # MDに適した計算モード
            show_progress=True,
            show_logger=True,
        )
    
        print(f"\nMD simulation finished.")
        print(f"Trajectory saved to: {OUTPUT_TRAJ}")
    
        # --- 6. ★最重要★ メモリの即時解放 ---
        print("シミュレーションが完了しました。メモリを圧迫する可能性があるため、")
        print("結果変数 `md_results` を削除し、メモリを強制解放します...")
        
        try:
            del md_results # トラジェクトリファイルさえあれば、これは不要な場合が多い
            gc.collect()
            print("メモリ解放が完了しました。")
        except NameError:
            print("`md_results` が見つかりませんでしたが、処理は完了しています。")



シミュレーション実行前にメモリを解放します...
メモリ解放完了。

Starting NVT MD simulation at 500.0 K...
Time step: 0.1 fs
Total steps: 400000 (for 40.0 ps)
Log/Traj interval: 1000 steps
z < 6.0 Å にある 465 個の原子を固定します。
システムを 500.0 K に初期化しました。

--- MDシミュレーションを開始します (400000 ステップ) ---


  0%|          | 0/400000 [00:00<?, ?it/s]

The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/structure/output/PVDF_interfaces/PVDF_on_Al2O3_cell_repeat_2x2x1_with_H2O_d2_0_final.traj.
Note: The max disk size of /home/jovyan is about 98G.
steps:     0  energy：80.21 eV/atom  total energy: 80.26 eV/atom  temperature: 483.34 K  volume: 12083 Ang^3  density: 5.715 g/cm^3


PFPAPIError: Invalid input value is detected: coordinates (positions) contains infinite values.
=== When reporting an error, please also share the data below: ===
----------------------------------------------------------------------------
time: 2025-11-14T11:09:55.535777+00:00
pid: 8729
code: INVALID_ARGUMENT
method: /pfp.Estimator/Estimate
details: Invalid=20input=20value=20is=20detected:=20coordinates=20(position=
s)=20contains=20infinite=20values.
exc: <_InactiveRpcError=20of=20RPC=20that=20terminated=20with:
=09status=20=3D=20StatusCode.INVALID_ARGUMENT
=09details=20=3D=20"Invalid=20input=20value=20is=20detected:=20coordinates=
=20(positions)=20contains=20infinite=20values."
=09debug_error_string=20=3D=20"UNKNOWN:Error=20received=20from=20peer=20=20=
{grpc_message:"Invalid=20input=20value=20is=20detected:=20coordinates=20(po=
sitions)=20contains=20infinite=20values.",=20grpc_status:3}"
>
notebook_id: 98ampuoye1y7dcec
metadata: [('client-process-id',=20'8729'),=20('n_atoms',=20'266

In [4]:
help(units)

Help on module ase.units in ase:

NAME
    ase.units - ase.units

DESCRIPTION
    Physical constants and units derived from CODATA for converting
    to and from ase internal units.

CLASSES
    builtins.dict(builtins.object)
        Units

    class Units(builtins.dict)
     |  Units(*args, **kwargs)
     |
     |  Dictionary for units that supports .attribute access.
     |
     |  Method resolution order:
     |      Units
     |      builtins.dict
     |      builtins.object
     |
     |  Methods defined here:
     |
     |  __init__(self, *args, **kwargs)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |
     |  ----------------------------------------------------------------------
     |  Data descriptors defined here:
     |
     |  __dict__
     |      dictionary for instance variables
     |
     |  __weakref__
     |      list of weak references to the object
     |
     |  ----------------------------------------------------------------------

In [2]:
pip install pfcc_extras

Looking in indexes: https://pypi.org/simple, http://pypi.artifact.svc:8080/simple
  Using cached http://pypi.artifact.svc:8080/simple/pfcc-extras/0.12.2/pfcc-extras-0.12.2.tar.gz (164 kB)
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached numpngw-0.1.4-py3-none-any.whl.metadata (14 kB)
  Using cached pymatgen_analysis_defects-2025.9.26-py3-none-any.whl.metadata (6.9 kB)
  Using cached pymatgen_io_validation-0.1.2-py3-none-any.whl.metadata (15 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached mako-1.3.10-py3-none-any.whl.metadata (2.9 kB)
  Using cached mp_pyrho-0.5.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached tifffile-2025.10.16-py3-